# Building a Chatbot that Doesn't Suck

In this notebook we'll build a RAG-based chatbot for a small furniture manufacturer in Oahu, Hawaii

## Set Up Authorization Tokens

In this notebook we'll use:

- [Jina Embeddings v2]()
- [Hugging Face Inference API]()

You'll need to get tokens for each of the above and enter them below.

In [34]:
from getpass import getpass

jinaai_api_key = getpass(prompt="Your Jina Embeddings API key: ")
hf_inference_api_key = getpass(prompt="Your Hugging Face Inference API key: ")

Your Jina Embeddings API key:  ········
Your Hugging Face Inference API key:  ········


In [35]:
# RAG dependencies
!pip install -q llama-index llama-index-llms-openai llama-index-embeddings-jinaai llama-index-llms-huggingface "huggingface_hub[inference]"


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


## Process data

We used GPT to generate some sample data for a fictitious small furniture maker in Oahu, Hawaii. This consists of four simple HTML pages:

- FAQ page
- Front page
- Contact page
- Product listings page

### Download Data

In [36]:
from glob import glob
import os
import subprocess

In [37]:
# cleanup from last run
!rm -rf data
!mkdir data

In [38]:
# download html files
!wget -q https://github.com/alexcg1/rag-chatbot/raw/main/notebook/data/faq.html --directory-prefix data/
!wget -q https://github.com/alexcg1/rag-chatbot/raw/main/notebook/data/front.html --directory-prefix data/
!wget -q https://github.com/alexcg1/rag-chatbot/raw/main/notebook/data/contact.html --directory-prefix data/
!wget -q https://github.com/alexcg1/rag-chatbot/raw/main/notebook/data/products.html --directory-prefix data/

In [39]:
# store html files in list
data_dir = "./data"
html_files = glob(f'{data_dir}/*.html')

### Convert to Markdown

HTML is a pain to break into chunks and unreliable for LLMs to parse. We'll convert it to [markdown]() to make things easier:

In [40]:
# convert html files to markdown for easier chunking
for filename in html_files:
  base_name = os.path.splitext(filename)[0]
  md_file = os.path.join(base_name + ".md")

  # Colab uses ancient pandoc, with different argument for markdown header style
  try:
    # colab pandoc
    subprocess.run(["pandoc", "--atx-headers", filename, "-o", md_file], check=True)
  except:
    # newer pandoc
    subprocess.run(["pandoc", "--markdown-headings=atx", filename, "-o", md_file], check=True)

md_files = glob(f'{data_dir}/*.md')

Unknown option --atx-headers.
Try pandoc --help for more information.
Unknown option --atx-headers.
Try pandoc --help for more information.
Unknown option --atx-headers.
Try pandoc --help for more information.
Unknown option --atx-headers.
Try pandoc --help for more information.


### Break Pages into Chunks

We'll make the data more digestible to our chatbot by breaking it into chunks:

In [65]:
# break markdown files into chunks
docs = []

for md_file in md_files:
    with open(md_file, 'r') as f:
          
        content = f.read()
        docs.append(content) # add full page
    
        content_chunks = content.split("\n#")
        docs.extend(content_chunks) # add individual sections

In [66]:
len(docs)

75

In [67]:
docs

["# FAQ - Nautical Furniture Hawaii\n\n## What types of wood do you use for your furniture?\n\nWe use sustainable native timbers sourced from Hawaii, such as Koa,\nMilo, and Kamani. These woods are known for their durability and\nbeautiful grain patterns.\n\n## Do you offer custom designs?\n\nYes, we specialize in custom nautical-themed furniture. Whether you have\nspecific dimensions or design ideas, we can work with you to create a\npiece that fits your needs.\n\n## Where are you located?\n\nWe are based in Oahu, Hawaii. Our workshop is located in Honolulu.\n\n## Do you ship internationally?\n\nCurrently, we primarily serve customers within Hawaii. Shipping options\noutside of Hawaii may be available upon request.\n\n## How do I care for nautical-themed furniture?\n\nRegular dusting and occasional polishing with a wood-friendly polish or\nwax will help maintain the natural beauty of your furniture. Avoid\nplacing furniture in direct sunlight or excessive humidity.\n\n## Can I visit y

## Build RAG system

### Access Jina Embeddings v2 via the LlamaIndex interface.

This code creates the LlamaIndex object that manages your connection to the Jina Embeddings v2 API.

The resulting object is held in the variable `jina_embedding_model`.


In [42]:
from llama_index.embeddings.jinaai import JinaEmbedding

jina_embedding_model = JinaEmbedding(
    api_key=jinaai_api_key,
    model="jina-embeddings-v2-base-en",
)

### Access the Mixtral Model via the HuggingFace Inference API

This code creates a holder for accessing the `mistralai/Mixtral-8x7B-Instruct-v0.1` model via the Hugging Face Inference API. The resulting object is held in the variable `mixtral_llm`.

In [43]:
from llama_index.llms.huggingface import HuggingFaceInferenceAPI

mixtral_llm = HuggingFaceInferenceAPI(
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1", token=hf_inference_api_key
)

/tmp/ipykernel_14734/4156006060.py:3: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPI. (Deprecated in favor of `HuggingFaceInferenceAPI` from `llama-index-llms-huggingface-api` which should be used instead.)
  mixtral_llm = HuggingFaceInferenceAPI(


### Convert Chunks To Be Suitable for LlamaIndex

In [44]:
from llama_index.core.readers import StringIterableReader
from llama_index.core.schema import Document

chunks = StringIterableReader().load_data(docs)

### Create a Service

The code creates a RAG service that has access to Jina Embeddings and Mixtral Instruct and stores it in the variable `service_context`.

In [45]:
from llama_index.core import ServiceContext

service_context = ServiceContext.from_defaults(
    llm=mixtral_llm, embed_model=jina_embedding_model
)

/tmp/ipykernel_14734/3144623853.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


### Build the Document Index

Next, we store the documents in LlamaIndex' `VectorStoreIndex`, generating embeddings with Jina Embeddings v2 model and using them as keys for retrieval.

**Note:** this may take several minutes.

In [50]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents=chunks, service_context=service_context
)

ConnectionError: HTTPSConnectionPool(host='api.jina.ai', port=443): Max retries exceeded with url: /v1/embeddings (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7b7fec3ae6f0>: Failed to resolve 'api.jina.ai' ([Errno -3] Temporary failure in name resolution)"))

### Prepare a Prompt Template

This is the prompt template that will be presented to Mixtral Instruct, with `{context_str}` and `{query_str}` replaced with the retrieved documents and your query respectively.

In [ ]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query. Please be brief, concise, and complete.\n"
    "If the context information does not contain an answer to the query, "
    "respond with \"I'm sorry, but we don't have any information about that. Please contact us on info@oahufurniture.com for more information.\"."
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt = PromptTemplate(qa_prompt_tmpl)

### Assemble the Full Query Engine

The query engine has three parts:

* `retriever` is the search engine that takes user requests and retrieves relevant documents from the vector store.
* `response_synthesizer` uses the prompt created above to join the retrieved documents and user request and passes them to the LLM, getting back its response.
* `query_engine` is a container object that holds the two together.

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    text_qa_template=qa_prompt,
    response_mode="compact",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

## Ask Some Questions

Let's ask some queries to see our chatbot in action:

In [ ]:
from IPython.display import HTML

def get_answer(question):
    result = query_engine.query(question)
    return HTML(result.response.strip())

In [ ]:
get_answer("How is a computer useful on a farm?")

In [ ]:
get_answer("What kind of furniture do you make?")

In [ ]:
get_answer("How much does your furniture cost?")

In [ ]:
get_answer("Can I see your furniture in person?")

In [ ]:
get_answer("What payment methods do you accept?")

In [ ]:
get_answer("What is your furniture made from?")

### Testing in different languages

In [ ]:
get_answer("你的家具是用什么材料制成的？")

In [ ]:
get_answer("Welche Zahlungsmethoden werden akzeptiert?")

### Ask Your Own Questions

Enter your question below and then hit enter to send. The question should be generated within a few seconds. Type "stop" to quit the question loop.

In [ ]:
while True:
    question = input("Please enter your question: ")
    if question.lower() == "stop":
        break
    answer = get_answer(question)
    print(answer)

## Set up API for external access

If you're running this locally in a Jupyter notebook (i.e. **_not_ Google Colab**) you can test the chatbot via a RESTful API and simple web interface:

In [ ]:
port = 5000

In [ ]:
!pip install -q fastapi uvicorn requests

In [ ]:
import os
import threading
import uvicorn
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# Enable CORS
origins = ["*"] # all origins

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

# Define FastAPI routes

@app.post('/')
async def chat_endpoint(request: Request):
    data = await request.json()  # Get JSON data from the request
    response_data = {
        "question": data["question"],
        "answer": get_answer(data["question"])
    }
    
    return response_data

@app.post('/shutdown')
async def shutdown():
    global server_running
    server_running = False
    def stop_uvicorn():
        uvicorn_server.should_exit = True
    threading.Thread(target=stop_uvicorn).start()
    return {"message": "Server shutting down..."}

# Start the FastAPI server in a new thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=port)

threading.Thread(target=run).start()

INFO:     127.0.0.1:35970 - "POST /shutdown HTTP/1.1" 200 OK


Exception in thread Thread-6 (stop_uvicorn):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/alexcg/work/repos/jina-alexcg/blog/chatsmith/rag-chatbot/notebook/env/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_14734/2761188623.py", line 48, in stop_uvicorn
NameError: name 'uvicorn_server' is not defined


INFO:     127.0.0.1:37164 - "POST /shutdown HTTP/1.1" 200 OK


Exception in thread Thread-7 (stop_uvicorn):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/alexcg/work/repos/jina-alexcg/blog/chatsmith/rag-chatbot/notebook/env/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_14734/2761188623.py", line 48, in stop_uvicorn
NameError: name 'uvicorn_server' is not defined


### Test in browser

We can open a simple HTML chatbot page for you to test out the chatbot

In [ ]:
!git clone https://github.com/alexcg1/rag-chatbot

In [ ]:
os.chdir("./rag-chatbot/web")

In [29]:
import http.server
import socketserver
import os

web_port = 8000

handler = http.server.SimpleHTTPRequestHandler

with socketserver.TCPServer(("", web_port), handler) as httpd:
    print("Serving at port", web_port)
    httpd.serve_forever()

KeyboardInterrupt: 

Now you can open your web browser to [http://localhost:8000](http://localhost:8000) to play with the chatbot in your browser.

To stop the web server, go to _Kernel_ -> _Interrupt Kernel_.

### Stop FastAPI Server

In [32]:
import requests

def stop_server():
    response = requests.post(f"http://localhost:{port}/shutdown")
    print(response.content)

In [33]:
stop_server()

b'{"message":"Server shutting down..."}'
